In [1]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method

"""

'\nPurpose: To decompose the multi-somas for splitting\nusing the new decomposition method\n\n'

In [3]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-19 18:32:50,445 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 18:32:50,447 - settings - Setting database.user to celiib
INFO - 2021-05-19 18:32:50,447 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 18:32:50,451 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-19 18:32:50,452 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 18:32:50,464 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-19 18:32:50,875 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 18:32:50,879 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 18:32:50,880 - settings - Setting database.user to celiib
INFO - 2021-05-19 18:32:50,880 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 18:32:50,881 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 18:32:50,881 - settings - Setting database.user to celiib
INFO - 2021-05-19 18:32:50,883 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 18:32:50,884 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-19 18:32:51,311 - settings - Setting enable_python_native_blobs to True


In [4]:
test_mode = True

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-19 18:32:53,697 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 18:32:53,698 - settings - Setting database.user to celiib
INFO - 2021-05-19 18:32:53,699 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 18:32:53,700 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-19 18:32:53,701 - settings - Setting database.user to celiib
INFO - 2021-05-19 18:32:53,702 - settings - Setting database.password to newceliipass
INFO - 2021-05-19 18:32:53,704 - settings - Setting enable_python_native_blobs to True


Sleeping 72 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-05-19 18:32:54,195 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)
temporary_folder = 'decimation_temp'
meshlab_scripts = {}

No port chosen so picked random port 7194


In [8]:
#so that it will have the adapter defined
from datajoint_utils import *

In [11]:
import numpy as np
import time
decimation_version = 0
decimation_ratio_global = 0.25

perform_mesh_segment_insert = False
verbose=True

from minfig.minnie65_config import external_decimated_mesh_path

@schema
class Decimation(dj.Computed):
#     definition = minnie.Decimation.describe(printout=False)


    #key_source = minnie.Mesh.proj() * (minnie.DecimationConfig & 'decimation_ratio=0.25') & minnie.MultiSomaProofread2.proj()
    #key_source = (du.version_to_segment_id_table() - (minnie.Decimation() & dict(decimation_ratio=decimation_ratio_global)).proj())
    key_source = (du.current_validation_segment_id_restriction - (minnie.Decimation() & dict(decimation_ratio=decimation_ratio_global)).proj()) & minnie.Mesh.proj()
    #key_source = minnie.Mesh() & dict(segment_id=864691136008592558)

    # Creates hf file at the proper location, returns the filepath of the newly created file
    @classmethod
    def make_file(cls, segment_id, version, decimation_ratio, vertices, faces):
        """Creates hf file at the proper location, returns the filepath of the newly created file"""

        assert vertices.ndim == 2 and vertices.shape[1] == 3
        assert faces.ndim == 2 and faces.shape[1] == 3

        filename = f'{segment_id}_{version}_{int(decimation_ratio*100):02}.h5'
        filepath = os.path.join(external_decimated_mesh_path, filename)
        with h5py.File(filepath, 'w') as hf:
            hf.create_dataset('segment_id', data=segment_id)
            hf.create_dataset('version', data=version)
            hf.create_dataset('decimation_ratio', data=float(decimation_ratio))
            hf.create_dataset('vertices', data=vertices)
            hf.create_dataset('faces', data=faces)

        return filepath

    @classmethod
    def make_entry(cls, segment_id, version, decimation_ratio, vertices, faces):
        key = dict(
            segment_id=segment_id,
            version=version,
            decimation_ratio=decimation_ratio,
            n_vertices=len(vertices),
            n_faces=len(faces)
        )

        filepath = cls.make_file(segment_id, version, decimation_ratio, vertices, faces)

        cls.insert1(dict(key, mesh=filepath), allow_direct_insert=True)

    

    def make(self, key):
        print(key)
        segment_id = key['segment_id']
        # -------- 1/27: Inserting the data into Mesh and Segment first ----- #
        if perform_mesh_segment_insert:
            #2) Manually add segmnet ids to segment tables
            minnie.Segment.insert1(dict(segment_id=segment_id),skip_duplicates=True)
            
            #3) Fill in the Mesh Table
            du.fill_from_ids(segment_ids=[segment_id])
            
            if verbose:
                print("Done Inserting Mesh")
        
        
        
        if "version" not in key.keys():
            version = decimation_version
        
        if "decimation_ratio" not in key.keys():
            decimation_ratio = decimation_ratio_global
            
        mesh = (minnie.Mesh & key).fetch1('mesh')
        
        
        print(f"Mesh size: n_vertices = {len(mesh.vertices)}, n_faces = {len(mesh.faces)}")

        if decimation_ratio not in meshlab_scripts:
            meshlab_scripts[decimation_ratio] = meshlab.Decimator(decimation_ratio, temporary_folder, overwrite=False)
        mls_func = meshlab_scripts[decimation_ratio]

        try:
            expected_filepath = os.path.join(external_decimated_mesh_path, f'{segment_id}_{version}.h5')
            if not os.path.isfile(expected_filepath):
                new_mesh, _path = mls_func(mesh.vertices, mesh.faces, segment_id)
                new_vertices, new_faces = new_mesh.vertices, new_mesh.faces

                self.make_entry(
                    segment_id=segment_id,
                    version=version,
                    decimation_ratio=decimation_ratio,
                    vertices=new_vertices,
                    faces=new_faces,
                    )
            else:
                print('File already exists.')
                with h5py.File(expected_filepath, 'r') as hf:
                    vertices = hf['vertices'][()].astype(np.float64)
                    faces = hf['faces'][()].reshape(-1, 3).astype(np.uint32)
                self.insert1(dict(key, n_vertices=len(vertices), n_faces=len(faces), mesh=expected_filepath), allow_direct_insert=True)
        except Exception as e:
            minnie.DecimationError.insert1(dict(key, log=str(e)))
            print(e)
            raise e
    

# Running the Populate

In [12]:
curr_table = (minnie.schema.jobs & "table_name='__decimation'")
curr_table#.delete()
#curr_table.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decimation,00377f5b109485e869a5defb2f6b9c46,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427072,2021-05-17 16:00:35
__decimation,0577fe53aed3f61fa48c14cace77c0d5,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.157,at-node30,1,427051,2021-05-17 16:00:17
__decimation,098e0be9fd9ace0ef11c71e33c872874,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.140,at-node13,1,427070,2021-05-17 15:54:37
__decimation,0b32188cc97c146c758bf0868d25e350,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.155,at-node28,1,427093,2021-05-17 16:00:28
__decimation,0b9a0dc68db9c46ed591d4d6fb9f428d,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.66.4.27,at-compute005,1,427063,2021-05-17 16:00:19
__decimation,17c785a4b4690cb9745c874d38ae6638,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.164,at-node37,1,427054,2021-05-17 16:00:19
__decimation,18cd69c4da59d8e5637c21fb14470057,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.133,at-node6,1,427056,2021-05-17 15:50:59
__decimation,1cf6ccb3ad6fd43fafb08cca317df795,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427078,2021-05-17 16:00:25
__decimation,1ee98a5dabebf5e40832306e5e0cf5cb,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.83,at-compute004,1,427082,2021-05-17 16:00:21
__decimation,20007fb0b96ab55246a54530e15b4f19,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.159,at-node32,1,427089,2021-05-17 16:00:21


In [13]:
import time
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decimation.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decimation.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for Decimation populate = {time.time() - start_time}")

INFO - 2021-05-19 18:33:56,056 - autopopulate - Found 4 keys to populate
INFO - 2021-05-19 18:33:56,068 - connection - Transaction started
INFO - 2021-05-19 18:33:56,069 - autopopulate - Populating: {'segment_id': 864691135307114822}


Populate Started
{'segment_id': 864691135307114822}
Mesh size: n_vertices = 4957944, n_faces = 9899465
xvfb-run -n 7194 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135307114822.off -o /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135307114822_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/decimation_meshlab_25595246.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135307114822.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135307114822_decimated.off


INFO - 2021-05-19 18:41:34,599 - connection - Transaction committed and closed.
INFO - 2021-05-19 18:41:34,608 - connection - Transaction started
INFO - 2021-05-19 18:41:34,610 - autopopulate - Populating: {'segment_id': 864691135373686345}


{'segment_id': 864691135373686345}
Mesh size: n_vertices = 6018109, n_faces = 12015956
xvfb-run -n 7194 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135373686345.off -o /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135373686345_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/decimation_meshlab_25595246.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135373686345.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691135373686345_decimated.off


INFO - 2021-05-19 18:51:05,909 - connection - Transaction committed and closed.
INFO - 2021-05-19 18:51:05,922 - connection - Transaction started
INFO - 2021-05-19 18:51:05,924 - autopopulate - Populating: {'segment_id': 864691136050851571}


{'segment_id': 864691136050851571}
Mesh size: n_vertices = 5286019, n_faces = 10555523
xvfb-run -n 7194 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691136050851571.off -o /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691136050851571_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/decimation_meshlab_25595246.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691136050851571.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Validation_Testing/v4_unseen/decimation_temp/neuron_864691136050851571_decimated.off


INFO - 2021-05-19 18:58:35,366 - connection - Transaction committed and closed.


Populate Done
Total time for Decimation populate = 1479.3286392688751
